# Covid-19 in Ireland
> Dashboard

- toc: false
- branch: master
- badges: false
- comments: false
- author: "Libre AI"
- categories: [ireland, covid19]

In [57]:
#hide
import altair as alt
import pandas as pd
import numpy as np

In [58]:
#hide
cases_ie = pd.read_csv('data/ie-covid19-ts.csv')
cases_it = pd.read_csv('data/it-covid19-ts.csv')
cases_es = pd.read_csv('data/es-covid19-ts.csv')

In [59]:
#hide
cases_ie.tail()

,date,country,cases,deaths,recovered,days_since_10
17,2020-03-18,ie,366,2,5,14
18,2020-03-19,ie,557,3,5,15
19,2020-03-20,ie,683,3,5,16
20,2020-03-21,ie,785,3,5,17
21,2020-03-22,ie,906,4,5,18


In [60]:
#hide
cases_ie[cases_ie['date'] == '2020-03-17']

,date,country,cases,deaths,recovered,days_since_10
16,2020-03-17,ie,292,2,5,13


In [61]:
#hide
# cases_ie.tail(), cases_it.tail(), cases_es.tail()

In [62]:
#hide

new_cases_ie = np.diff(cases_ie['cases'])
new_cases_ie = np.hstack((np.array([0]), new_cases_ie))

new_cases_it = np.diff(cases_it['cases'])
new_cases_it = np.hstack((np.array([0]), new_cases_it))

new_cases_es = np.diff(cases_es['cases'])
new_cases_es = np.hstack((np.array([0]), new_cases_es))

In [63]:
#hide
#figures from https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population
population_in_m_ie=4.9
population_in_m_it=60.2
population_in_m_es=47.1

In [64]:
#hide

source_ie = pd.DataFrame({
    'country': 'Ireland',
    'date': cases_ie['date'],
    'days_since_10_cases': cases_ie['days_since_10'],
    'confirmed_cases': cases_ie['cases'],
    'confirmed_cases_per_million': np.round(cases_ie['cases']/population_in_m_ie),
    'new_cases': new_cases_ie,
    'deaths': cases_ie['deaths']
})

source_it = pd.DataFrame({
    'country': 'Italy',
    'date': cases_it['date'],
#     'days_since_10_cases': cases_it['days_since_10'][cases_it['days_since_10'] < 18],
    'days_since_10_cases': cases_it['days_since_10'],
    'confirmed_cases': cases_it['cases'],
    'confirmed_cases_per_million': np.round(cases_it['cases']/population_in_m_it),
    'new_cases': new_cases_it,
    'deaths': cases_it['deaths']
})

source_es = pd.DataFrame({
    'country': 'Spain',
    'date': cases_es['date'],
#     'days_since_10_cases': cases_es['days_since_10'][cases_es['days_since_10'] < 18],
    'days_since_10_cases': cases_es['days_since_10'],
    'confirmed_cases': cases_es['cases'],
    'confirmed_cases_per_million': np.round(cases_es['cases']/population_in_m_es),
    'new_cases': new_cases_es,
    'deaths': cases_es['deaths']
})


In [65]:
#hide

cases_ie_plot = alt.Chart(source_ie).mark_circle(
    color='green',
    opacity=0.7).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases:Q', scale=alt.Scale(type='sqrt')),
    size=alt.Size('new_cases:Q', title="Number of new cases in Ireland"),
    tooltip=['country', 'date:T', 'confirmed_cases', 'new_cases', 'deaths']
)#.interactive()

cases_ie_plot

alt.Chart(...)

In [66]:
#hide

cases_it_plot = alt.Chart(source_it).mark_line(
    color='blue',
    opacity=0.5,
    strokeDash=[4, 4],
    strokeWidth=3,
    point=True
).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases:Q', scale=alt.Scale(type='sqrt')),
#     size='new_cases:Q',
    tooltip=['country', 'date:T', 'confirmed_cases', 'new_cases', 'deaths'],
    color=alt.Color('country', scale=alt.Scale(range=['blue'], domain=['Italy']), title="Country")
)



cases_it_plot

alt.Chart(...)

In [67]:
#hide

cases_es_plot = alt.Chart(source_es).mark_line(
    color='red',
    opacity=0.5,
    strokeDash=[4,4],
    strokeWidth=3,
    point=True
).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases:Q', scale=alt.Scale(type='sqrt')),
#     size='new_cases:Q',
    tooltip=['country', 'date:T', 'confirmed_cases', 'new_cases', 'deaths'],
    color=alt.Color('country', scale=alt.Scale(range=['red'], domain=['Spain']), title="")
)

cases_es_plot

alt.Chart(...)

# Number of persons tested positive for Covid-19 in Ireland
The figure below shows the total number of persons who tested positive for Covid-19 (_cases_) in Ireland, Italy, and Spain. The `x` axis corresponds to the number of days since at least 10 cases were detected. This allow us to compare Ireland's current situation to the observed in Italy and Spain earlier. The comparison using Italy and Spain is because those countries are still in a critical situation fighting the pandemia, and no country would like to be the next one in such situation. 

For Ireland, the size of the circles for each day denotes the number of new persons confirmed infected that day.

**If we consider only the absolute numbers, we can see that the confirmed cases for Covid-19 in Ireland are below the ones observed in Italy and Spain at early stages. However, the populations of the three countries are very different and we have to account for that as well.**

In [68]:
#hide_input

alt.layer(cases_it_plot, cases_es_plot, cases_ie_plot).properties(
    width=800, height=600
).resolve_scale(
    color='independent',
    shape='independent'
)

alt.LayerChart(...)

# Number of Covid-19 cases per 1M population
The pupulation of Ireland, Italy, and Spain is as follows (in millions) [\[source\]](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population):

- **Ireland** : 4.9M
- **Italy** : 60.2M
- **Spain** : 47.1M

If we plot a figure dividing the total number of cases reported daily by the population (in millions) we can find how many persons are infected per 1 million, which give us a normalized value we could use to understand the severity of the outbreak.

### The figure below shows that in fact Ireland has more cases per million that the most affected countries in Europe starting on March 16th, 2020, and growing quickly.

In [69]:
#hide

cases_per_million_ie_plot = alt.Chart(source_ie).mark_circle(
    color='green',
    opacity=0.7).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases_per_million:Q', scale=alt.Scale(type='sqrt')),
    size=alt.Size('new_cases:Q', title="Number of new cases in Ireland"),
    tooltip=['country', 'date:T', 'confirmed_cases:Q', 'confirmed_cases_per_million:Q', 'new_cases', 'deaths']
)#.interactive()

cases_per_million_ie_plot

alt.Chart(...)

In [70]:
#hide

cases_per_million_it_plot = alt.Chart(source_it).mark_line(
    color='blue',
    opacity=0.5,
    strokeDash=[4, 4],
    strokeWidth=3,
    point=True
).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases_per_million:Q', scale=alt.Scale(type='sqrt')),
#     size='new_cases:Q',
    tooltip=['country', 'date:T', 'confirmed_cases:Q', 'confirmed_cases_per_million:Q', 'new_cases', 'deaths'],
    color=alt.Color('country', scale=alt.Scale(range=['blue'], domain=['Italy']), title="Country")
)

cases_per_million_it_plot

alt.Chart(...)

In [71]:
#hide

cases_per_million_es_plot = alt.Chart(source_es).mark_line(
    color='red',
    opacity=0.5,
    strokeDash=[4,4],
    strokeWidth=3,
    point=True
).encode(
    x='days_since_10_cases:Q',
#     y='confirmed_cases:Q',
    y=alt.Y('confirmed_cases_per_million:Q', scale=alt.Scale(type='sqrt')),
#     size='new_cases:Q',
    tooltip=['country', 'date:T', 'confirmed_cases:Q', 'confirmed_cases_per_million:Q', 'new_cases', 'deaths'],
    color=alt.Color('country', scale=alt.Scale(range=['red'], domain=['Spain']), title="")
)

cases_per_million_es_plot

alt.Chart(...)

In [72]:
#hide_input

alt.layer(cases_per_million_ie_plot, cases_per_million_it_plot, cases_per_million_es_plot).properties(
    width=800, height=600
).resolve_scale(
    color='independent',
    shape='independent'
)

alt.LayerChart(...)

---
### Data sournces:
- Government of Ireland [https://www.gov.ie/en/news/7e0924-latest-updates-on-covid-19-coronavirus/]
- 2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE [https://github.com/CSSEGISandData/COVID-19]
